In [2]:
import pandas as pd
import numpy as np
import csv

'''
let A = the first input table (or ideally, the larger one)
let B = the second input table (or ideally, the smaller one)
let jA = the join column ID of table A
let jB = the join column ID of table B
let MB = a multimap for mapping from single values to multiple rows of table B (starts out empty)
let C = the output table (starts out empty)

for each row b in table B:
   place b in multimap MB under key b(jB)

for each row a in table A:
   for each row b in multimap MB under key a(jA):
      let c = the concatenation of row a and row b
      place row c in table C
'''

TableR = pd.read_csv("r1.csv")
TableS = pd.read_csv("s1.csv")

print(TableR)
print(TableS)

     ID  FirstName
0     0  Stephanie
1     1      Kerry
2     2      Helen
3     3   Jennifer
4     5    Brandon
..  ...        ...
95  191     Gerald
96  192       Mary
97  194     Tamara
98  196      Debra
99  198    Jeffrey

[100 rows x 2 columns]
     ID  LastName
0     0   Huffman
1     7    Tilley
2    11  Franklin
3    13   Ramirez
4    15     Gregg
..  ...       ...
95  194     Toppa
96  195   Cameron
97  196     Jones
98  198     Davis
99  199     Bryan

[100 rows x 2 columns]


In [3]:
#HashJoin
TableT = {}

for person in TableS["ID"]:
    if person not in TableT:
        TableT[person] = [TableS[TableS["ID"] == person].values.flatten().tolist()]
    else:
        TableT[person] = TableT[person].append( TableS[TableS["ID"] == person].values.flatten().tolist())
#(s, r) for r in table2 for s in h[r[index2]]

for index, R in TableR.iterrows():
    for S in TableT[R[0]]:
        S.append(R.values.flatten().tolist())
        
print(TableT)

KeyError: 1

In [4]:
# 2-phase Track-Join
# Initialize Table Tr|s

TableTrs = []
sent_to_R = []
sent_to_S = []

def send_to_t(index, table):
    print('%s sent %d to T' %(table, index))
    TableTrs.append([index, table])
    
def send_to_s(index, payload):
    print('%s sent %d to S' %(payload.values.flatten().tolist(), index))
    sent_to_S.append([index, payload])

def send_to_r(index, table):
    print('%s sent %d to R' %(table, index))
    sent_to_R.append([index, table])

# ->ProcessR

print(' ---> Process R ')

TableTr = {}

for index, person in TableR.iterrows():
    if person['ID'] not in TableTr:
        send_to_t(person['ID'], 'R')
        TableTr[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTr[person['ID']] = [TableTr[person['ID']], person.values.flatten().tolist()]


print(TableTr)
print(TableTrs)
print('---Barrier ProcessR----- \n ---> Process S ')

# ->ProcessS

TableTs = {}

for index, person in TableS.iterrows():
    if person['ID'] not in TableTs:
        send_to_t(person['ID'], 'S')
        TableTs[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTs[person['ID']] = [TableTs[person['ID']], person.values.flatten().tolist()]


print(TableTs)
print(TableTrs)
print('---Barrier Process S----- \n ---> Process T resumes')

# ->ProcessT after barrier

TableTrs_cpy = TableTrs
for row in TableTrs:
    if row[1] == 'R':
        for row2 in TableTrs_cpy:
            if row2[1] == 'S' and row[0] == row2[0]:
                send_to_r(row2[0], 'T')
print(sent_to_R)
print('---Ended Process T----- \n ---> Process R resumes')

# ->ProcessR after barrier

for row in sent_to_R:
    for index, row_val in TableR[TableR['ID'] == row[0]].iterrows():
        send_to_s(row[0], row_val)

print('---Ended Process R----- \n ---> Process S resumes')

# ->ProcessS after barrier

for row in sent_to_S:
    for index, row_val in TableS[TableS['ID'] == row[0]].iterrows():
        print(str(row_val.values.flatten().tolist()) + str(row[1].values.flatten().tolist()))
print('---Ended Process S-----')

 ---> Process R 
R sent 0 to T
R sent 1 to T
R sent 2 to T
R sent 3 to T
R sent 5 to T
R sent 7 to T
R sent 8 to T
R sent 12 to T
R sent 13 to T
R sent 14 to T
R sent 15 to T
R sent 16 to T
R sent 17 to T
R sent 18 to T
R sent 19 to T
R sent 21 to T
R sent 22 to T
R sent 25 to T
R sent 26 to T
R sent 29 to T
R sent 30 to T
R sent 31 to T
R sent 35 to T
R sent 36 to T
R sent 39 to T
R sent 40 to T
R sent 41 to T
R sent 44 to T
R sent 47 to T
R sent 48 to T
R sent 49 to T
R sent 57 to T
R sent 58 to T
R sent 62 to T
R sent 64 to T
R sent 65 to T
R sent 66 to T
R sent 67 to T
R sent 69 to T
R sent 70 to T
R sent 74 to T
R sent 76 to T
R sent 77 to T
R sent 78 to T
R sent 79 to T
R sent 89 to T
R sent 90 to T
R sent 91 to T
R sent 92 to T
R sent 95 to T
R sent 96 to T
R sent 97 to T
R sent 98 to T
R sent 99 to T
R sent 100 to T
R sent 106 to T
R sent 109 to T
R sent 110 to T
R sent 113 to T
R sent 116 to T
R sent 120 to T
R sent 122 to T
R sent 127 to T
R sent 128 to T
R sent 129 to T
R se

In [5]:
# 2-phase Track-Join
# Initialize Table Tr|s

TableTrs = []
sent_to_R = []
sent_to_S = []

def send_to_t(index, table):
    print('%s sent %d to T' %(table, index))
    TableTrs.append([index, table])
    
def send_to_s(index, payload):
    print('%s sent %d to S' %(payload.values.flatten().tolist(), index))
    sent_to_S.append([index, payload])

def send_to_r(index, table):
    print('%s sent %d to R' %(table, index))
    sent_to_R.append([index, table])

# ->ProcessR

print(' ---> Process R ')

TableTr = {}

for index, person in TableR.iterrows():
    if person['ID'] not in TableTr:
        send_to_t(person['ID'], 'R')
        TableTr[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTr[person['ID']] = [TableTr[person['ID']], person.values.flatten().tolist()]


print(TableTr)
print(TableTrs)
print('---Barrier ProcessR----- \n ---> Process S ')

# ->ProcessS

TableTs = {}

for index, person in TableS.iterrows():
    if person['ID'] not in TableTs:
        send_to_t(person['ID'], 'S')
        TableTs[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTs[person['ID']] = [TableTs[person['ID']], person.values.flatten().tolist()]


print(TableTs)
print(TableTrs)
print('---Barrier Process S----- \n ---> Process T resumes')

# ->ProcessT after barrier

TableTrs_cpy = TableTrs
for row in TableTrs:
    if row[1] == 'R':
        for row2 in TableTrs_cpy:
            if row2[1] == 'S' and row[0] == row2[0]:
                send_to_r(row2[0], 'T')
print(sent_to_R)
print('---Ended Process T----- \n ---> Process R resumes')

# ->ProcessR after barrier

for row in sent_to_R:
    for index, row_val in TableR[TableR['ID'] == row[0]].iterrows():
        send_to_s(row[0], row_val)

print('---Ended Process R----- \n ---> Process S resumes')

# ->ProcessS after barrier

for row in sent_to_S:
    for index, row_val in TableS[TableS['ID'] == row[0]].iterrows():
        print(str(row_val.values.flatten().tolist()) + str(row[1].values.flatten().tolist()))
print('---Ended Process S-----')

 ---> Process R 
R sent 0 to T
R sent 1 to T
R sent 2 to T
R sent 3 to T
R sent 5 to T
R sent 7 to T
R sent 8 to T
R sent 12 to T
R sent 13 to T
R sent 14 to T
R sent 15 to T
R sent 16 to T
R sent 17 to T
R sent 18 to T
R sent 19 to T
R sent 21 to T
R sent 22 to T
R sent 25 to T
R sent 26 to T
R sent 29 to T
R sent 30 to T
R sent 31 to T
R sent 35 to T
R sent 36 to T
R sent 39 to T
R sent 40 to T
R sent 41 to T
R sent 44 to T
R sent 47 to T
R sent 48 to T
R sent 49 to T
R sent 57 to T
R sent 58 to T
R sent 62 to T
R sent 64 to T
R sent 65 to T
R sent 66 to T
R sent 67 to T
R sent 69 to T
R sent 70 to T
R sent 74 to T
R sent 76 to T
R sent 77 to T
R sent 78 to T
R sent 79 to T
R sent 89 to T
R sent 90 to T
R sent 91 to T
R sent 92 to T
R sent 95 to T
R sent 96 to T
R sent 97 to T
R sent 98 to T
R sent 99 to T
R sent 100 to T
R sent 106 to T
R sent 109 to T
R sent 110 to T
R sent 113 to T
R sent 116 to T
R sent 120 to T
R sent 122 to T
R sent 127 to T
R sent 128 to T
R sent 129 to T
R se